# PETS CLASSIFICATION

## Libraries and extensions

Lo primero que debemos hacer antes de comenzar con la manipulacion de nuestros datos es importar todas las librerias que vayamos a utlizar.

In [12]:
from os.path import isfile, join, exists
from skimage import data, exposure
from os import listdir
import os
import torch
import shutil
from torchvision.datasets import ImageFolder
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from torch.utils.data import random_split
from sklearn.model_selection import train_test_split
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

## Transformación y preparación del conjunto de datos

Para la organización de las imágenes hemos decidido separar en dos carpetas (una para perros y otra para gatos) todas las imágenes. 

In [13]:
# Ruta al directorio principal con imágenes de gatos
gatos_directory = "images/images"

# Iterar sobre todas las imágenes de gatos
for filename in os.listdir(gatos_directory):
    if filename.endswith(".jpg"):
        # Obtener la primera letra del nombre de la raza
        primera_letra = filename.split("_")[0][0]

        # Determinar la carpeta de destino según la primera letra
        carpeta_destino = os.path.join(gatos_directory, "gatos" if primera_letra.isupper() else "perros")

        # Crear la carpeta de destino si no existe
        os.makedirs(carpeta_destino, exist_ok=True)

        # Crear la ruta completa a la imagen original y a la nueva ubicación
        original_path = os.path.join(gatos_directory, filename)
        nueva_ruta = os.path.join(carpeta_destino, filename)

        # Mover la imagen a la carpeta correspondiente
        shutil.move(original_path, nueva_ruta)

print("Imágenes organizadas según si es gato o perro.")

Imágenes organizadas según si es gato o perro.


# YOLO


In [2]:
!pip install -U ultralytics
%pip install tqdm


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


In [3]:
# Model
model_yolo = torch.hub.load('ultralytics/yolov5', 'yolov5n', pretrained=True)  # Carrega online

Using cache found in C:\Users\marcc/.cache\torch\hub\ultralytics_yolov5_master
C:\Users\marcc\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
YOLOv5  2023-12-18 Python-3.8.5 torch-2.0.1 CPU

Fusing layers... 
YOLOv5n summary: 213 layers, 1867405 parameters, 0 gradients
Adding AutoShape... 


In [14]:
results = model_yolo("images/images/gatos/Abyssinian_110.jpg") 
results.pandas().xyxy[0]

,xmin,ymin,xmax,ymax,confidence,class,name
0,7.081909,0.0,384.59079,310.551605,0.44604,15,cat


In [15]:
# Rutas a las carpetas con imágenes de perros y gatos
path_gatos = "images/images/gatos/"
path_perros = "images/images/perros/"

# Obtén la lista de archivos en cada carpeta y combínalas en una lista única
images_gatos = [os.path.join(path_gatos, img) for img in os.listdir(path_gatos) if img.endswith(".jpg")]
images_perros = [os.path.join(path_perros, img) for img in os.listdir(path_perros) if img.endswith(".jpg")]

# Combina ambas listas
images = images_gatos + images_perros

# Dividir los datos en conjuntos de entrenamiento y prueba
train_images, test_images = train_test_split(images, test_size=0.2, random_state=42)

# Imprimir información
print("Número de imágenes en el conjunto de entrenamiento:", len(train_images))
print("Número de imágenes en el conjunto de prueba:", len(test_images))

Número de imágenes en el conjunto de entrenamiento: 5912
Número de imágenes en el conjunto de prueba: 1478


In [16]:
def evaluar_conjunto(dataset, detect_fn):
    correct_counter = 0
    total_samples = len(dataset)

    for img_path in tqdm(dataset):
        results = model_yolo(img_path)  # Realiza la inferencia utilizando YOLO
        #results.pandas().xyxy[0]
        if detect_fn(results.xyxy[0]):
            correct_counter += 1

    accuracy = correct_counter / total_samples
    return accuracy

# Evaluar sobre el conjunto de prueba combinado
total_test_samples = len(test_images)
accuracy_total = evaluar_conjunto(test_images, lambda det: any(det[5] in [16, 15] and det[4] >= 0.2 for det in det))

print(f"Accuracy total en el conjunto de prueba: {accuracy_total * 100:.2f}%")

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1478/1478 [07:29<00:00,  3.29it/s]

Accuracy total en el conjunto de prueba: 73.48%


## Creación de nuestra propia red 
Ahora pasaremos a crear nuestra propia red para clasificar perros y gatos


#### División de Conjunto de Datos en Entrenamiento y Prueba:

En aprendizaje automático, es fundamental evaluar el rendimiento de un modelo en datos no utilizados durante el entrenamiento. La división del conjunto de datos en conjuntos de entrenamiento y prueba permite esta evaluación. A continuación, se explica el código utilizado:


`DataLoader` simplifica el proceso de carga y gestión de datos durante el entrenamiento de modelos de aprendizaje automático, proporcionando una interfaz fácil de usar y eficiente para trabajar con conjuntos de datos.

In [17]:
# Ruta del directorio de datos
data_root = r"images/images"

# Transformaciones para las imágenes
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# Crear una instancia de ImageFolder
dataset = ImageFolder(root=data_root, transform=transform)

# Porcentaje de datos para el conjunto de prueba
test_size = 0.2

# Calcular el número de muestras para el conjunto de prueba
test_size = int(test_size * len(dataset))

# Calcular el número de muestras para el conjunto de entrenamiento
train_size = len(dataset) - test_size

# Dividir el conjunto de datos en conjuntos de entrenamiento y prueba
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Tamaño del lote (batch size) que deseas utilizar
batch_size = 32

# DataLoader para el conjunto de entrenamiento
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# DataLoader para el conjunto de prueba
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Imprimir información
print("Número de imágenes en el conjunto de entrenamiento:", len(train_dataset))
print("Número de imágenes en el conjunto de prueba:", len(test_dataset))


Número de imágenes en el conjunto de entrenamiento: 5912
Número de imágenes en el conjunto de prueba: 1478


#### Arquitectura de nuestra red
En este punto crearemos la arquitectura de nuestra red

In [18]:
class CustomNet(nn.Module):
    def __init__(self):
        super(CustomNet, self).__init__()
        # Capa convolucional 1
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        # Capa convolucional 2
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        # Capa de pooling
        self.pool = nn.MaxPool2d(2, 2)
        # Capa completamente conectada 1
        self.fc1 = nn.Linear(64 * 56 * 56, 128)  # Ajusta según tu nuevo tamaño de imagen
        # Capa de Dropout para regularización
        self.dropout = nn.Dropout(0.5)  # Puedes ajustar la probabilidad de dropout
        # Capa completamente conectada 2 (salida)
        self.fc2 = nn.Linear(128, 2)  # 2 clases: perro y gato

    def forward(self, x):
        # Aplicar capas convolucionales y de pooling
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 56 * 56)  # Ajusta según tu nuevo tamaño de imagen
        x = F.relu(self.fc1(x))
        x = self.dropout(x)  # Aplicar Dropout
        x = self.fc2(x)
        return F.softmax(x, dim=1)


Una vez que hemos creado la arquitectura de nuestra red pasaremos a crear los métodos que entrenaran y testearan dicha arquitectura. Estas funciones se utilizan en el bucle de entrenamiento para actualizar los parámetros del modelo y evaluar su rendimiento, respectivamente.

In [19]:
def train(model, device, train_loader, optimizer, epoch, log_interval=100, verbose=True):
    pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad) # !!!
    print("Trainable Parameters: ", pytorch_total_params)
    model.train()
    loss_v = 0

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % log_interval == 0 and verbose:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
        
        loss_v += loss.item()

    loss_v /= len(train_loader.dataset)
    print('\nTrain set: Average loss: {:.4f}\n'.format(loss_v))

    return loss_v


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

    return test_loss

#### Entrenamiento y evaluacion de la red
Se realiza un bucle a lo largo de las épocas. En cada época, se entrena el modelo en el conjunto de entrenamiento y se evalúa en el conjunto de prueba. Los resultados se almacenan en arreglos para análisis posterior.

En la primera prueba que realizaremos utilizaremos el optimizador *Adam* y un *learning reate* de 0.1

In [23]:
use_cuda = torch.cuda.is_available()
torch.manual_seed(33)

device = torch.device("cuda" if use_cuda else "cpu")

epochs = 15
lr = 1e-1

model = CustomNet().to(device)

optimizer = optim.Adam(model.parameters(), lr=lr)

# Guardar los valores de pérdida promedio de cada iteración (época)
train_l = np.zeros((epochs))
test_l = np.zeros((epochs))

# Bucle de entrenamiento
for epoch in range(epochs):
    train_l[epoch] = train(model, device, train_loader, optimizer, epoch)
    test_l[epoch]  = test(model, device, test_loader)

Trainable Parameters:  25709314
Train Epoch: 0 [0/5912 (0%)]	Loss: 0.679923
Train Epoch: 0 [3200/5912 (54%)]	Loss: 0.625762

Train set: Average loss: 0.0200


Test set: Average loss: 0.6319, Accuracy: 1007/1478 (68%)

Trainable Parameters:  25709314
Train Epoch: 1 [0/5912 (0%)]	Loss: 0.594512
Train Epoch: 1 [3200/5912 (54%)]	Loss: 0.563262

Train set: Average loss: 0.0200


Test set: Average loss: 0.6319, Accuracy: 1007/1478 (68%)

Trainable Parameters:  25709314
Train Epoch: 2 [0/5912 (0%)]	Loss: 0.719512
Train Epoch: 2 [3200/5912 (54%)]	Loss: 0.719512

Train set: Average loss: 0.0200


Test set: Average loss: 0.6319, Accuracy: 1007/1478 (68%)

Trainable Parameters:  25709314
Train Epoch: 3 [0/5912 (0%)]	Loss: 0.688262
Train Epoch: 3 [3200/5912 (54%)]	Loss: 0.625762

Train set: Average loss: 0.0200


Test set: Average loss: 0.6319, Accuracy: 1007/1478 (68%)

Trainable Parameters:  25709314
Train Epoch: 4 [0/5912 (0%)]	Loss: 0.719512
Train Epoch: 4 [3200/5912 (54%)]	Loss: 0.750762

Tra

Ahora podemos trazar un gráfico para evaluar como ha ido el proceso de entrenamiento:

In [24]:
plt.title("Resultats de l'entrenament")
plt.plot(range(1, (epochs + 1)), train_l,  c="red", label="train")
plt.plot(range(1,  (epochs + 1)), test_l,  c="green", label="test")
plt.legend();

A continuacion probaaremos a volver a entrenar nuestra red pero cambiando algunos de los parametros.

In [ ]:
use_cuda = torch.cuda.is_available()
torch.manual_seed(33)

device = torch.device("cuda" if use_cuda else "cpu")

epochs = 15
lr = 0.1

model = CustomNet().to(device)

optimizer = optim.SGD(model.parameters(), lr=lr)

# Guardar los valores de pérdida promedio de cada iteración (época)
train_l = np.zeros((epochs))
test_l = np.zeros((epochs))

# Bucle de entrenamiento
for epoch in range(epochs):
    train_l[epoch] = train(model, device, train_loader, optimizer, epoch)
    test_l[epoch]  = test(model, device, test_loader)

Trainable Parameters:  25709314
Train Epoch: 0 [0/5912 (0%)]	Loss: 0.679923


In [ ]:
plt.title("Resultats de l'entrenament")
plt.plot(range(1, (epochs + 1)), train_l,  c="red", label="train")
plt.plot(range(1,  (epochs + 1)), test_l,  c="green", label="test")
plt.legend();

## Clasificacion de razas de perros y gatos

Para solucionar este problema haremos uso de nuestra propia red pero modificandola ligeramente para que pueda clasificar mas clases.

#### Preprocesing 
Primero de todo lanzaremos un script para subdividir las carpetas de las dos especies que tenemos en mas carpetas internas para cada raza.

In [22]:
def dividir_por_raza(type):
    # Ruta al directorio principal con imágenes de gatos
    gatos_directory = "images/images/"+type

    # Iterar sobre todas las imágenes de gatos
    for filename in os.listdir(gatos_directory):
        if filename.endswith(".jpg"):
            # Extraer el nombre de la raza de la imagen
            raza, _ = os.path.splitext(filename.split("_")[0])

            # Crear la ruta completa a la imagen original y a la nueva ubicación
            original_path = os.path.join(gatos_directory, filename)
            nueva_carpeta = os.path.join(gatos_directory, raza)

            # Crear la carpeta para la raza si no existe
            os.makedirs(nueva_carpeta, exist_ok=True)

            # Mover la imagen a la carpeta correspondiente
            nueva_ruta = os.path.join(nueva_carpeta, filename)
            shutil.move(original_path, nueva_ruta)

    print("Imágenes organizadas por raza.")
    
dividir_por_raza("perros")
dividir_por_raza("gatos")

Imágenes organizadas por raza.
Imágenes organizadas por raza.
